Data loading

In [6]:
import json
json_file_path = "val_file.json"

with open(json_file_path, "r") as f:
    test_data = json.load(f)

comp_utterances_test, comp_labels_test = [], []

for i in range(len(test_data)):
    comp_utterances_test += test_data[i]["utterances"]
    comp_labels_test += test_data[i]["emotions"]

In [7]:
pip install -U sentence-transformers

  Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)


In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [10]:
label_map = {"anger": 0, "fear": 1, "disgust": 2, "sadness": 3, "joy": 4, "surprise": 5, "neutral": 6}
test_embeddings=model.encode(comp_utterances_test)
test_labels_encoded = [label_map[label] for label in comp_labels_test]
test_embeddings_tensor = torch.tensor(test_embeddings,device='cuda')
test_labels_tensor = torch.tensor(test_labels_encoded,device='cuda')

# Create a PyTorch dataset
test_dataset = TensorDataset(test_embeddings_tensor, test_labels_tensor)

In [11]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)  # Take the output of the last timestep
        return out


In [12]:
device=torch.device('cuda')

In [19]:
# Hyperparameters
input_size = 384  # Size of word embeddings
hidden_size = 128  # Number of units in the LSTM hidden state
num_layers = 3  # Number of LSTM layers
output_size = len(list(label_map.keys()))  # Number of output classes
learning_rate = 0.001
num_epochs = 10

# Initialize LSTM model
m1 = LSTMModel(input_size, hidden_size, num_layers, output_size)
m1.load_state_dict(torch.load("Task_1_M1.pth", map_location=device))
m1.to(device)

LSTMModel(
  (lstm): LSTM(384, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=7, bias=True)
)

In [20]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder

# Define RNN model
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out)  # Take the output of the last timestep
        return out


In [21]:
# Hyperparameters
input_size = 384  # Size of word embeddings
hidden_size = 128  # Number of units in the LSTM hidden state
num_layers = 3  # Number of LSTM layers
output_size = len(list(label_map.keys()))  # Number of output classes
learning_rate = 0.001
num_epochs = 10

# Initialize RNN model
m2= RNNModel(input_size, hidden_size, num_layers, output_size)
m2.load_state_dict(torch.load("Task_1_M2.pth", map_location=device))
m2.to(device)

RNNModel(
  (rnn): RNN(384, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=7, bias=True)
)

Evaluation

In [22]:
#Model M1
# Evaluation
test_predictions=[]
m1.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(len(test_dataset)):
        inputs=test_embeddings_tensor[i].unsqueeze(0)
        labels=test_labels_tensor[i].unsqueeze(0)
        outputs = m1(inputs)
        _, predicted = torch.max(outputs.data, 1)
        #print(type(predicted))
        #print(predicted.item())
        #break
        test_predictions.append(predicted.item())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy on val data: {accuracy:.2f}')

Accuracy on val data: 0.96


In [ ]:
pred_emots=test_predictions


In [ ]:
json_file_path = "val_file.json"

with open(json_file_path, "r") as f:
    val_data = json.load(f)

keys = list(val_data[0].keys())
for i in range(len(val_data[0])):
    print(keys[i],":", val_data[0][keys[i]])
    print(len(val_data[0][keys[i]]))
    print("\n")

In [ ]:
with open("chr_dict.json", "r") as f:
    chr_dict = json.load(f)

with open("emt_dict.json", "r") as f:
    emt_dict = json.load(f)

In [ ]:
comp_data_list_val = []
cnt = 0
tot_cnt = 0
for inst in range(len(val_data)):
    # Define the speaker dict
    speaker_dict = {}
    chr_list = val_data[inst]["speakers"]
    tot_cnt += len(chr_list)

    # Create empty speaker dict
    for j in chr_list:
        speaker_dict[chr_dict[j]] = {}

    # Number of utterances and fill in speaker dict
    for utt in range(len(val_data[inst]["utterances"])):
        speaker_dict[chr_dict[chr_list[utt]]][utt] = emt_dict[val_data[inst]["emotions"][utt]]

    # Assign index 1 as trigger list
    data_list = [speaker_dict,val_data[inst]["triggers"]]

    # Add it to the complete list
    comp_data_list_val.append(data_list)

    cnt += 1

print(tot_cnt)

In [ ]:
with open("comp_data_list_val.json", "w") as json_file:
    json.dump(comp_data_list_val, json_file)

In [ ]:
# utts are the indexes of emotes that belong to this particular character
# emotes are the subsection of pred_emotes relevant to this scene
def return_flips (utt,emotes):
    flips = []
    for i in range(1,len(utt)):
        if emotes[utt[i-1]] != emotes[utt[i]]:
            flips.append(("u"+str(utt[i-1]),"u"+str(utt[i])))
    return flips

In [ ]:
all_flips = []
cur_pointer = 0
for i in range(len(comp_data_list_val)):
    diag_len = len(comp_data_list_val[i][1]) # How many emotions to pick up == num of targets given at the end
    emots = pred_emots[cur_pointer:cur_pointer + diag_len]

    # For every character involved in the scene
    char_diag_dict = {} # maps character to its flips
    for char in comp_data_list_val[i][0]:
        utt = [int(j) for j in comp_data_list_val[i][0][char]]
        char_diag_dict[char] = return_flips(utt,emots)

    all_flips.append(char_diag_dict)

    cur_pointer += diag_len

In [ ]:
with open("all_flips_M1.json", "w") as json_file:
    json.dump(all_flips, json_file)

In [24]:
#Model M2
# Evaluation
test_predictions=[]
m2.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(len(test_dataset)):
        inputs=test_embeddings_tensor[i].unsqueeze(0)
        labels=test_labels_tensor[i].unsqueeze(0)
        outputs = m2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        #print(type(predicted))
        #print(predicted.item())
        #break
        test_predictions.append(predicted.item())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy on val data: {accuracy:.2f}')

Accuracy on val data: 0.89


In [ ]:
pred_emots=test_predictions

In [ ]:
json_file_path = "val_file.json"

with open(json_file_path, "r") as f:
    val_data = json.load(f)

keys = list(val_data[0].keys())
for i in range(len(val_data[0])):
    print(keys[i],":", val_data[0][keys[i]])
    print(len(val_data[0][keys[i]]))
    print("\n")

In [ ]:
with open("chr_dict.json", "r") as f:
    chr_dict = json.load(f)

with open("emt_dict.json", "r") as f:
    emt_dict = json.load(f)

In [ ]:
comp_data_list_val = []
cnt = 0
tot_cnt = 0
for inst in range(len(val_data)):
    # Define the speaker dict
    speaker_dict = {}
    chr_list = val_data[inst]["speakers"]
    tot_cnt += len(chr_list)

    # Create empty speaker dict
    for j in chr_list:
        speaker_dict[chr_dict[j]] = {}

    # Number of utterances and fill in speaker dict
    for utt in range(len(val_data[inst]["utterances"])):
        speaker_dict[chr_dict[chr_list[utt]]][utt] = emt_dict[val_data[inst]["emotions"][utt]]

    # Assign index 1 as trigger list
    data_list = [speaker_dict,val_data[inst]["triggers"]]

    # Add it to the complete list
    comp_data_list_val.append(data_list)

    cnt += 1

print(tot_cnt)

In [ ]:
with open("comp_data_list_val.json", "w") as json_file:
    json.dump(comp_data_list_val, json_file)

In [ ]:
# utts are the indexes of emotes that belong to this particular character
# emotes are the subsection of pred_emotes relevant to this scene
def return_flips (utt,emotes):
    flips = []
    for i in range(1,len(utt)):
        if emotes[utt[i-1]] != emotes[utt[i]]:
            flips.append(("u"+str(utt[i-1]),"u"+str(utt[i])))
    return flips

In [ ]:
all_flips = []
cur_pointer = 0
for i in range(len(comp_data_list_val)):
    diag_len = len(comp_data_list_val[i][1]) # How many emotions to pick up == num of targets given at the end
    emots = pred_emots[cur_pointer:cur_pointer + diag_len]

    # For every character involved in the scene
    char_diag_dict = {} # maps character to its flips
    for char in comp_data_list_val[i][0]:
        utt = [int(j) for j in comp_data_list_val[i][0][char]]
        char_diag_dict[char] = return_flips(utt,emots)

    all_flips.append(char_diag_dict)

    cur_pointer += diag_len

In [ ]:
with open("all_flips_M2.json", "w") as json_file:
    json.dump(all_flips, json_file)